In [14]:
import requests
from bs4 import BeautifulSoup
import csv

In [32]:
import requests
from bs4 import BeautifulSoup
import csv

url = "https://www.worlddata.info/alliances/un-united-nations.php"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm tất cả các thẻ <tr>
    tr_elements = soup.find_all('tr')

    # Lặp qua từng thẻ <tr>
    data = []
    for tr in tr_elements:
        # Tìm tất cả các thẻ <td> trong từng thẻ <tr>
        td_elements = tr.find_all('td')

        # Tạo danh sách để lưu trữ dữ liệu
        row_data = []

        for td in td_elements:
            # Tìm thẻ <a> trong từng thẻ <td>
            a_element = td.find('a')
            if a_element:
                # Lấy đường link từ thẻ <a> nếu tồn tại
                link = a_element['href']
                row_data.append(link)

                # Truy cập tiếp vào đường link và lấy thông tin từ trang mới
                subpage_url = 'https://www.worlddata.info/'+link
                response_subpage = requests.get(subpage_url)
                
                if response_subpage.status_code == 200:
                    soup_subpage = BeautifulSoup(response_subpage.text, 'html.parser')
                    # Tìm thẻ div có chữ "Capital:"
                    div_capital = soup_subpage.find('div', string='Capital:')
                    
                    if div_capital:
                        # Lấy thẻ div có class "floater" đứng sau thẻ div "Capital:"
                        div_floater = div_capital.find_next('div', class_='floater')
                        
                        if div_floater:
                            capital = div_floater.text.strip()
                            row_data.append(capital)

        # Thêm dữ liệu của hàng vào danh sách chính
        data.append(row_data)

# Lưu dữ liệu vào file CSV
csv_filename = 'output_with_capitals.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print(f'Data with capitals has been saved to {csv_filename}')


Data with capitals has been saved to output_with_capitals.csv


In [38]:
import requests
from bs4 import BeautifulSoup
import csv

url = "https://www.worlddata.info/alliances/un-united-nations.php"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm tất cả các thẻ <tr>
    tr_elements = soup.find_all('tr')

    # Lặp qua từng thẻ <tr>
    data = []
    country_header_added = False

    for tr in tr_elements:
        # Tìm tất cả các thẻ <td> trong từng thẻ <tr>
        td_elements = tr.find_all('td')

        # Tạo danh sách để lưu trữ dữ liệu
        row_data = []

        for td in td_elements:
            # Tìm thẻ <a> trong từng thẻ <td>
            a_element = td.find('a')
            if a_element:
                # Lấy nội dung text từ thẻ <a> nếu tồn tại
                text_content = a_element.text.strip()
                row_data.append(text_content)

        # Nếu chưa thêm header "Country", thêm vào đầu danh sách
        if not country_header_added:
            data.append(['Country'])
            country_header_added = True

        # Thêm dữ liệu của hàng vào danh sách chính
        data.append(row_data)

# Lưu dữ liệu vào file CSV
csv_filename = 'countries.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print(f'Data has been saved to {csv_filename}')


Data has been saved to countries.csv


In [42]:
import csv
import requests

# Function to get latitude and longitude from Geocode API
def get_lat_lon_from_geocode_api(capital):
    api_key = "65b12ffc9e1eb780254126kdcc393e8"
    base_url = f"https://geocode.maps.co/search?q={capital}&api_key={api_key}"

    response = requests.get(base_url)
    if response.status_code == 200:
        try:
            json_data = response.json()
            lat = json_data[0]['lat']
            lon = json_data[0]['lon']
            return lat, lon
        except (KeyError, IndexError):
            return None, None
    else:
        return None, None

# Read the countries.csv file
country_filename = 'countries.csv'
capitals_data = []

with open(country_filename, 'r', newline='', encoding='utf-8') as country_file:
    country_reader = csv.reader(country_file)
    header = next(country_reader)  # Read the header row
    country_name_index = header.index('Country')
    capital_index = header.index('Capital')

    for row in country_reader:
        if len(row) >= max(country_name_index, capital_index) + 1:  # Ensure the row has enough columns
            country_name = row[country_name_index]
            capital = row[capital_index]

            # Get latitude and longitude using the Geocode API
            lat, lon = get_lat_lon_from_geocode_api(capital)

            # Append the data to the list
            capitals_data.append([country_name, capital, lat, lon])

# Save the data to a new CSV file
output_filename = 'capitals_with_lat_lon.csv'
with open(output_filename, 'w', newline='', encoding='utf-8') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(['Country', 'Capital', 'Latitude', 'Longitude'])  # Header row
    csv_writer.writerows(capitals_data)

print(f'Data with latitude and longitude has been saved to {output_filename}')


Data with latitude and longitude has been saved to capitals_with_lat_lon.csv
